In [1]:
from models import Category,Product,database
import peewee as pe

# C

In [ ]:
Category.create(name='VIP', description='Unique things') # method

In [ ]:
with database:
    Category(name = 'Standalone').save() # Object

In [ ]:
with database:
    q= Category.insert(name = 'Others') # Querry
    q.execute()

In [ ]:
categories_data = [
    dict(name='Books', description='Papers books.'),
    dict(name='Electronic Books', description='Web books.'),
    dict(name='Mashines', description='Printers,scanets, ect.'),
    dict(name='Pens', description='Pencils and pens')
]
with database: 
    for line in categories_data:
        Category.create(**line) # create in loop

In [ ]:
categories_data = [
    dict(name='Books2', description='Papers books v2.'),
    dict(name='Electronic_Books2', description='Web books v2.'),
    dict(name='Mashines2', description='Printers,scanets, ect v2.'),
    dict(name='Pens2', description='Pencils and pens v2')
]
with database:
    q= Category.insert_many(categories_data,fields=[Category.name,Category.description]) #querry 
    q.execute()

In [ ]:
categories_data = [
    dict(name='Books3', description='Papers books v3.'),
    dict(name='Electronic Books3', description='Web books v3.'),
    dict(name='Mashines3', description='Printers,scanets, ect v3.'),
    dict(name='Pens3', description='Pencils and pens v3')
]
with database.atomic():
    for line in categories_data:
        Category.create(**line) # create in loop as one transaction

In [ ]:
with database:
    books = Category.get(Category.name == 'Books')
    Product.create(name='The economist', price=100, category=books)


In [ ]:
names= ['Journal','Thermodinamics','Kinetic']
with database:
    books = Category.get(Category.name == 'Books')
    for name in names:
        Product.create(name=name, price=100, category=books)


In [ ]:
names = ['Scanner', 'Printer', 'Cartridge', 'Pad']

[{'name': name, 'category': 1, 'price': 20-len(name)} for name in names]


In [ ]:
with database:
    maschines = Category.get(name='Mashines')
    data = [{
        'name': name, 'category': maschines, 'price': 20*10 - len(name)*10
    } for name in names]
    Product.insert_many(
        data,
        fields=[Product.name, Product.category, Product.price],
    ).execute()


# R

In [ ]:
#querry automatic use q.execute() in for
with database:
    q = Category.select()

for line in q: 
    print(line.id,line.name,)

In [ ]:
# strange behavior
with database:
    p = Category.get(Category.id>3)
    
print(p.id,p.name)
# for i in p:
#     print(p.id)

In [ ]:
# 1+n problem selection
with database:
    q = Product.select()
    q = q.execute()
print(q)
print(list(q)[5].id)


for line in q:
    print('{0:2} {1:.<30} {2:>4.0f} | {3: <10}'.format(line.id,
                                                    line.name,
                                                    line.price,
                                                    line.category.name)) # 1+n problem

In [3]:
# Solving 1+n problem selection
with database:
    q = Product.select(Product.id,Product.name,Product.price,Category.name).join(Category)
    print(q)
    q = q.execute()
print(list(q)[5].id)

for line in q:
    # print(line.name)
    # print('{0:2} {1:.<30} {2:>4.0f} | {3: <10}'.format(line.id,
    #                                                 line.name,
    #                                                 line.price,
    #                                                 line.category.name))

SELECT `t1`.`id`, `t1`.`name`, `t1`.`price`, `t2`.`name` FROM `Products` AS `t1` INNER JOIN `Categories` AS `t2` ON (`t1`.`category_id` = `t2`.`id`)


ProgrammingError: (1146, "Table 'orm.products' doesn't exist")

In [ ]:
# Solving 1+n w/o object
with database:
    q = Product.select(Product,Category).join(Category)
    


In [6]:
# 1+n count group by
with database:
    for cat in Category.select():
        print(cat.name,Product.select().where(Product.category_id == cat).count())

# with database:
#     for cat in Category.select():
#         for i in Product.select().where(Product.category_id == cat):
#             print(i.name)
#         print('{0:.^20}'.format('Empty'))

VIP 0
VIP 0
Standalone 0
Othersadde 0
Books 4
Electronic 0
Mashines 8
Pens 0
Books2 0
Electronic 0
Mashines2 0
Pens2 0
Books2 0
Electronic 0
Mashines2 0
Pens2 0


In [27]:
# solving 1+n count group by
with database:
    q = (Category
        .select(Category.name, pe.fn.count(Product.id).alias('num'))
        .join(Product)
        .group_by(Category))
    print(q)
q = q.execute()
print(list(q)[0].name)

for line in q:
    print(line.name,line.num)

SELECT `t1`.`name`, count(`t2`.`id`) AS `num` FROM `Categories` AS `t1` INNER JOIN `product` AS `t2` ON (`t2`.`category_id` = `t1`.`id`) GROUP BY `t1`.`id`, `t1`.`name`, `t1`.`description`
Books


In [13]:
with database:
    q= Category.select()
    print(q)
    q.prefetch(Product)
print(q[0].name)

for line in q:
    print(line.name)
    for prod in line.fk_category:
        print(' *',prod.name)

SELECT `t1`.`id`, `t1`.`name`, `t1`.`description` FROM `Categories` AS `t1`
VIP


In [7]:
# Dicts(), namedtuples(), tuples(), dicts() 
with database:
    q= Category.select().dicts() 
list(q)

[{'id': 1, 'name': 'VIP', 'description': 'Unique things'},
 {'id': 2, 'name': 'VIP', 'description': 'Unique things'},
 {'id': 3, 'name': 'Standalone', 'description': ''},
 {'id': 4, 'name': 'Othersadde', 'description': ''},
 {'id': 5, 'name': 'Books', 'description': 'Papers books.'},
 {'id': 6, 'name': 'Electronic', 'description': 'Web books.'},
 {'id': 7, 'name': 'Mashines', 'description': 'Printers,scanets, ect.'},
 {'id': 8, 'name': 'Pens', 'description': 'Pencils and pens'},
 {'id': 9, 'name': 'Books2', 'description': 'Papers books v2.'},
 {'id': 10, 'name': 'Electronic', 'description': 'Web books v2.'},
 {'id': 11, 'name': 'Mashines2', 'description': 'Printers,scanets, ect v2.'},
 {'id': 12, 'name': 'Pens2', 'description': 'Pencils and pens v2'},
 {'id': 13, 'name': 'Books2', 'description': 'Papers books v2.'},
 {'id': 14, 'name': 'Electronic', 'description': 'Web books v2.'},
 {'id': 15, 'name': 'Mashines2', 'description': 'Printers,scanets, ect v2.'},
 {'id': 16, 'name': 'Pens2'

# U

In [ ]:
with database:
    cat = Category.get_by_id(4) 
    cat.name +='added'
    cat.save()  #Update object
    print(cat.name)

In [ ]:
with database:
    q= Category.update(name = 'unknown').where(Category.id>16) # update querry
    q.execute()

# D

In [ ]:
with database:
    q= Category.delete().where(Category.name =='unknown' ) #querry
    q.execute()

# M

# Tests

In [ ]:
with database:
    q1 = Category.select()
    q2= q1.where(Category.id >5)
    q3 =q2.order_by(Category.name)
    for line in q3:
        print(line.id,line.name)

In [ ]:
with database:
    q = Category.select()
    q2 = q.where(Category.id>3)
    q3 = q2.execute()
print(type(q))
print(q)
print('*'*40)
print(type(q2))
print(q2)
print('*'*40)
print(type(q3))
print(q3)

In [ ]:
with database:
    q = Category.select()
    q2 = q.name